# Loading in the data

In [23]:
import json
import re
def extract_json_formatted( response):
    #remove trailing and leading json formatting
    response_text = clean_json_string(response)
    try:
        return json.loads(response_text)
    except json.JSONDecodeError:
        print(f"Skipping invalid JSON in response: {response_text}")
        return None
    
def clean_json_string(json_string):
        pattern = r'^```json\s*(.*?)\s*```$'
        cleaned_string = re.sub(pattern, r'\1', json_string, flags=re.DOTALL)
        return cleaned_string.strip()

def read_jsonl_and_append_data(file_path, data_storage):
    """
    Reads a JSONL file and appends parsed data to the provided data storage list.

    Parameters:
    - file_path: Path to the JSONL file.
    - data_storage: List to append the parsed data to.
    """
    with open(file_path, 'r') as file:
        for line in file:
            # Parse the JSON line
            sample = json.loads(line)
            # Append a structured object to the data storage
            data_storage.append({
                'data': json.loads(sample['user_input']),
                'labels': extract_json_formatted(sample['response'])
            })

# Example usage:
data_array = []
file_path = 'classification_agendapunt_openai_dataset.jsonl'
read_jsonl_and_append_data(file_path, data_array)

file_path = 'classification_agendapunt_openai_dataset_meta.jsonl'
read_jsonl_and_append_data(file_path, data_array)


In [32]:
print("length of data_array: ", len(data_array))
print("data_array[0]: ", data_array[0])

length of data_array:  4993
data_array[0]:  {'data': {'uri': 'https://data.gent.be/id/agendapunten/22.0215.3724.2149', 'title': '2022_RMW_00022 - Samenwerkingsovereenkomst betreffende Vlaamse Housing First middelen voor CAW Oost-Vlaanderen - Goedkeuring', 'description': 'Aan de raad voor maatschappelijk welzijn wordt gevraagd goedkeuring te verlenen aan de Samenwerkingsovereenkomst met Centrum voor Algemeen Welzijnswerk vzw, Visserij 153, 9000 Gent betreffende Vlaamse Housing First middelen voor CAW Oost-Vlaanderen.\r\n'}, 'labels': {'classification': {'stadsbestuur': ['samenwerkingsovereenkomst'], 'samenleven, welzijn en gezondheid': ['dak- en thuisloosheid', 'welzijnswerk']}}}


# Loading the taxonomy

In [43]:
taxonomy = json.loads(open(r'..\source\taxonomy_agendapunten.json').read())
classes = list(taxonomy.keys())
print("classes: ", classes)

classes:  ['burgerzaken', 'stadsbestuur', 'cultuur, sport en vrije tijd', 'mobiliteit en openbare werken', 'groen en milieu', 'onderwijs en kinderopvang', 'samenleven, welzijn en gezondheid', 'werken en ondernemen', 'wonen en (ver)bouwen']


# Creating the spacy training and testing data

In [25]:
# print distribution of labels

label_distribution = {}

for data in data_array:
    labels = data['labels']["classification"]
    if labels:
        for label in labels:
            if label in label_distribution:
                label_distribution[label] += 1
            else:
                label_distribution[label] = 1

                
print("length of label_distribution (top-levels): ", len(label_distribution))
display("label_distribution (top-levels): ", label_distribution)


length of label_distribution (top-levels):  10


'label_distribution (top-levels): '

{'stadsbestuur': 2662,
 'samenleven, welzijn en gezondheid': 241,
 'wonen en (ver)bouwen': 782,
 'groen en milieu': 215,
 'mobiliteit en openbare werken': 693,
 'cultuur, sport en vrije tijd': 607,
 'werken en ondernemen': 59,
 'onderwijs en kinderopvang': 229,
 'burgerzaken': 91,
 'veiligheid en preventie': 1}

In [47]:
import spacy
from spacy.tokens import DocBin
from spacy.training import Example
import random

nlp = spacy.blank("nl")  # Assuming Dutch language model

# Assuming 'classes' list is defined and populated with all possible classes

spacy_data = []
for data in data_array:
    text = data['data']['title'] + " " + data['data']['description']
    labels = list(data['labels']['classification'].keys())  # Adjust according to your data structure
    # Initialize all classes to 0
    cats = {label: 0 for label in classes}
    # Set the class to 1 if it's present in the labels
    for label in labels:
        if label in classes:
            cats[label] = 1
    # Check if at least one class is assigned
    if any(cats.values()):
        spacy_data.append((text, {"cats": cats}))

# Shuffle the data to ensure random distribution
random.shuffle(spacy_data)

# Split the data (e.g., 80% train, 20% test)
split_ratio = 0.8
split_index = int(len(spacy_data) * split_ratio)
train_data = spacy_data[:split_index]
test_data = spacy_data[split_index:]

# Function to create a DocBin from data
def create_docbin(data, nlp):
    doc_bin = DocBin(attrs=["LEMMA", "ENT_IOB", "ENT_TYPE"], store_user_data=True)
    for text, annotations in data:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        doc_bin.add(example.reference)
    return doc_bin

# Create DocBin for both train and test data
train_doc_bin = create_docbin(train_data, nlp)
test_doc_bin = create_docbin(test_data, nlp)

# Save the DocBins to files
train_doc_bin.to_disk("./spacy_textcat_train.spacy")
test_doc_bin.to_disk("./spacy_textcat_test.spacy")

print("Training and testing datasets created and saved.")

Training and testing datasets created and saved.


In [46]:
# Load the test DocBin
test_doc_bin = DocBin().from_disk("./spacy_textcat_test.spacy")

# Load the docs from the DocBin
test_docs = list(test_doc_bin.get_docs(nlp.vocab))

# Print 5 samples
for doc in test_docs[:5]:
    print(doc.text)
    print("Categories:", doc.cats)
    print("---")

2021_CBS_01153 - 2021/00226M - Aktename melding voor het bouwen van een veranda langs Berkenstraat 3 in 3950 Bocholt - Goedkeuring Aktename melding voor het bouwen van een veranda langs de Berkenstraat
Categories: {'burgerzaken': 0, 'stadsbestuur': 0, 'cultuur, sport en vrije tijd': 0, 'mobiliteit en openbare werken': 0, 'groen en milieu': 0, 'onderwijs en kinderopvang': 0, 'samenleven, welzijn en gezondheid': 0, 'werken en ondernemen': 0, 'wonen en (ver)bouwen': 1}
---
2022_CBS_00895 - Bestelbonnen 2022 week 19: goedkeuring  
Categories: {'burgerzaken': 0, 'stadsbestuur': 1, 'cultuur, sport en vrije tijd': 0, 'mobiliteit en openbare werken': 0, 'groen en milieu': 0, 'onderwijs en kinderopvang': 0, 'samenleven, welzijn en gezondheid': 0, 'werken en ondernemen': 0, 'wonen en (ver)bouwen': 0}
---
11.  Onteigeningsplan Grotestraat (vanaf Warvinge tot grens met Zuienkerke) - Definitief onteigeningsbesluit 
Categories: {'burgerzaken': 0, 'stadsbestuur': 0, 'cultuur, sport en vrije tijd': 0,

# Generate the config file and train the model using the datasets

In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg

In [ ]:
!python -m train config.cfg --paths.train ./spacy_textcat_train.spacy --paths.dev ./spacy_textcat_test.spacy --output ./output

# Inference

In [ ]:
import spacy

nlp = spacy.load("../robbert-2023-abb-agendapunten-classifier/model-best")

text = """
2021_CBS_01153 - 2021/00226M - Aktename melding voor het bouwen van een veranda langs Berkenstraat 3 in 3950 Bocholt - Goedkeuring Aktename melding voor het bouwen van een veranda langs de Berkenstraat
"""
doc = nlp(text)

print(doc.cats)
